In [ ]:
import requests

URL = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/"

r = requests.get(url=URL)

In [ ]:
companies_cvm_codes = [
    "22349",
    "25208",
    "25011",
    "25518",
    "8133",
    "10456",
    "24694",
    "4669",
    "22055",
    "024848",
    "26506",
    "21440",
    "10324",
    "4146",
]
type_docs = ["BPA", "BPP", "DFC_MD", "DFC_MI", "DMPL", "DRE", "DVA"]
first_year = 2010
last_year = 2023

In [ ]:
from datetime import datetime

if first_year and last_year:
    year_range = [x for x in range(first_year, last_year + 1)]
elif first_year:
    last_year_tmp = datetime.now().year
    year_range = [x for x in range(first_year, last_year_tmp + 1)]
else:
    first_year_tmp = 2010
    year_range = [x for x in range(first_year_tmp, last_year + 1)]


year_filter = [f"{year}.zip" for year in year_range]

year_filter

In [ ]:
# !pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup

html = BeautifulSoup(r.text, "html.parser")

files_list = [x.get("href") for x in html.find("pre").find_all("a") if x.get("href")[-8:] in year_filter]

files_list

In [ ]:
full_links = [f"{URL}{file_}" for file_ in files_list]
full_links

In [ ]:
import zipfile
import io

TMP_FILES_PATH = "tmp_files/"

tmp_file = requests.get(full_links[0], stream=True)
unzipped = zipfile.ZipFile(io.BytesIO(tmp_file.content))

filtered_files = [file_ for file_ in unzipped.namelist() for file_type in type_docs if file_type in file_]

for selected_file in filtered_files:
    unzipped.extract(selected_file, path=TMP_FILES_PATH)

In [ ]:
import pandas as pd

FILE = filtered_files[0]

FULL_COLUMNS = [
    "CNPJ_CIA",
    "CD_CVM",
    "DT_REFER",
    "DT_INI_EXERC",
    "DT_FIM_EXERC",
    "DENOM_CIA",
    "VERSAO",
    "GRUPO_DFP",
    "MOEDA",
    "ESCALA_MOEDA",
    "ORDEM_EXERC",
    "CD_CONTA",
    "DS_CONTA",
    "VL_CONTA",
    "COLUNA_DF",
]

df = pd.read_csv(f"{TMP_FILES_PATH}{FILE}", sep=";", decimal=",", encoding="Latin1")
df["VL_CONTA"] = df["VL_CONTA"].astype("float64")

df

In [ ]:
missing_columns = [x for x in FULL_COLUMNS if x not in df.columns]

if len(missing_columns) > 0:
    for col in missing_columns:
        df[col] = None

df = df[FULL_COLUMNS]

if all(df["DT_INI_EXERC"].isna()):
    df.loc[:, "DT_INI_EXERC"] = df["DT_REFER"].unique()[0][:5] + "01-01"

df["source_file"] = FILE

df

In [ ]:
if companies_cvm_codes:
    df = df[df["CD_CVM"].isin(companies_cvm_codes)]

df

In [ ]:
AVAILABLE_DOCS = ["BPA", "BPP", "DFC_MD", "DFC_MI", "DMPL", "DRE", "DVA"]

AVAILABLE_FORMATS = ["ind", "con"]

In [ ]:
# get_dfp_data(companies_cvm_codes = None, first_year = 2010, last_year = datetime.now().year,
# type_docs = ['BPA', 'BPP', 'DRE'], type_format = ['con', 'ind'], clean_data = True)
#

# download_read_dfp_zip_file(url_in, companies_cvm_codes, type_format, type_docs, clean_data)

# read_dfp_csv(file_in, clean_data)

# clean_dfp_itr_data(df_in, file_in)

In [ ]:
import os

# os.remove()

os.remove(f"{TMP_FILES_PATH}{FILE}")

In [1]:
# import sys

# sys.path.append("..")

companies_cvm_codes = [22349, 25208, 25011, 25518, 8133, 10456, 24694, 4669, 22055, 24848, 26506, 21440, 10324, 4146]
type_docs = ["BPA", "BPP", "DFC_MD", "DFC_MI", "DMPL", "DRE", "DVA"]
first_year = 2010
last_year = 2023


groups = [
    "DF Consolidado - Balanço Patrimonial Passivo",
    "DF Consolidado - Balanço Patrimonial Ativo",
    "DF Consolidado - Demonstração do Fluxo de Caixa (Método Indireto)",
    "DF Consolidado - Demonstração do Resultado",
    "DF Consolidado - Demonstração das Mutações do Patrimônio Líquido",
]

accounts = [
    "3.03",
    "3.04",
    "3.04.01",
    "3.04.02",
    "3.04.03",
    "3.04.04",
    "3.04.05",
    "6.01.01.04",
    "6.01.01.02",
    "3.05",
    "3.01",
    "3.02",
]
keep_columns = ["DT_FIM_EXERC", "DENOM_CIA", "VL_CONTA", "CD_CONTA", "DS_CONTA"]

incompatible_accounts = {"accounts": ["6.01.01.04", "6.01.01.02"], "ds_account": "depreciação|depreciações"}

cd_conta_map_dict = {
    "6.01.01.04": "Depreciações e Amortizações_2",
    "6.01.01.02": "Depreciações e Amortizações_1",
    "3.03": "Resultado Bruto",
    "3.04.01": "Despesas de Vendas",
    "3.04.02": "Despesas Gerais e Administrativas",
    "3.04.03": "Perdas por não recuperabilidade de Ativos",
    "3.04.04": "Outras Receitas Operacionais",
    "3.04.05": "Outras Despesas Operacionais",
    "3.05": "EBIT",
    "3.01": "Receita de Venda de Bens/Serviços",
    "3.02": "Custo de Bens/Serviços",
    "3.04": "Despesas/Receitas Operacionais (totais)",
}

update_company_name = {"RESTOQUE COMÉRCIO E CONFECÇÕES DE ROUPAS S.A.": "VESTE S.A. ESTILO"}

In [2]:
from src.peers_comparison.cvm_data_retriever import DFPRetriever
import numpy as np

retriever = DFPRetriever()

cvm_df = retriever.get_dfp_data(
    companies_cvm_codes=companies_cvm_codes, first_year=first_year, last_year=last_year, type_docs=type_docs
)

In [3]:
df = cvm_df[(cvm_df["VL_CONTA"] < 0) | (cvm_df["VL_CONTA"] > 1)].copy()

df = df[(df["GRUPO_DFP"].isin(groups)) & (df["CD_CONTA"].isin(accounts))][keep_columns]

df["remove"] = np.where(
    df["CD_CONTA"].isin(incompatible_accounts["accounts"]),
    np.where(df["DS_CONTA"].str.lower().str.contains(incompatible_accounts["ds_account"]), 0, 1),
    0,
)

df = (
    df[df["remove"] == 0][["DT_FIM_EXERC", "DENOM_CIA", "VL_CONTA", "CD_CONTA"]]
    .replace({"CD_CONTA": cd_conta_map_dict})
    .replace({"DENOM_CIA": update_company_name})
)

df.head()

,DT_FIM_EXERC,DENOM_CIA,VL_CONTA,CD_CONTA
5427,2010-12-31,GUARARAPES CONFECCOES S.A.,108898.0,Depreciações e Amortizações_2
9697,2010-12-31,VESTE S.A. ESTILO,13172.0,Depreciações e Amortizações_1
11325,2010-12-31,ALPARGATAS S.A.,50771.0,Depreciações e Amortizações_1
11558,2010-12-31,MARISA LOJAS S.A.,103355.0,Depreciações e Amortizações_1
12767,2010-12-31,KARSTEN S.A.,5579.0,Depreciações e Amortizações_2


In [4]:
df_pivot = df.pivot(index=["DT_FIM_EXERC", "DENOM_CIA"], columns="CD_CONTA", values="VL_CONTA").reset_index().fillna(0)

df_pivot.head()

CD_CONTA,DT_FIM_EXERC,DENOM_CIA,Custo de Bens/Serviços,Depreciações e Amortizações_1,Depreciações e Amortizações_2,Despesas Gerais e Administrativas,Despesas de Vendas,Despesas/Receitas Operacionais (totais),EBIT,Outras Despesas Operacionais,Outras Receitas Operacionais,Perdas por não recuperabilidade de Ativos,Receita de Venda de Bens/Serviços,Resultado Bruto
0,2010-12-31,ALPARGATAS S.A.,-1217751.0,50771.0,0.0,-120391.0,-546472.0,-697180.0,324225.0,-46457.0,25362.0,0.0,2239156.0,1021405.0
1,2010-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,-339884.0,0.0,0.0,-45679.0,-96597.0,-138821.0,92820.0,0.0,3455.0,0.0,571525.0,231641.0
2,2010-12-31,GUARARAPES CONFECCOES S.A.,-1115401.0,0.0,108898.0,-363957.0,-679362.0,-1037539.0,455044.0,0.0,5780.0,0.0,2607984.0,1492583.0
3,2010-12-31,KARSTEN S.A.,-230175.0,0.0,5579.0,-26464.0,-76622.0,-94873.0,32883.0,0.0,8213.0,0.0,357931.0,127756.0
4,2010-12-31,LOJAS RENNER S.A.,-1209459.0,75737.0,0.0,-211764.0,-641702.0,-1137405.0,404474.0,-209378.0,13611.0,-88172.0,2751338.0,1541879.0


In [5]:
from src.peers_comparison.sheets_data import get_sheets_data

sheets_data = get_sheets_data()

sheets_data.head()

,DT_FIM_EXERC,DENOM_CIA,EBIT,Calculo_Margem_Bruta,Depreciações e Amortizações_2,Depreciações e Amortizações_1,Receita de Venda de Bens/Serviços,Custo de Bens/Serviços,Resultado Bruto,Despesas/Receitas Operacionais (totais),Despesas de Vendas,Despesas Gerais e Administrativas,Outras Receitas Operacionais,Outras Despesas Operacionais
0,2019-12-31,AMMO.VAREJO,15063.47676,0.488221,4575.119930,0.0,320885.0801,-164222.0922,156662.9879,-182044.7860,-129405.0,-43578.0,991.366277,-10053.15228
1,2020-12-31,AMMO.VAREJO,11067.79964,0.514921,6676.017490,0.0,431143.2593,-209138.3288,222004.9304,-226511.9087,-188015.0,-23646.0,1964.925644,-16815.83439
2,2021-12-31,AMMO.VAREJO,20743.71900,0.533911,8516.201330,0.0,495250.3615,-230830.9053,264419.4562,-262201.3709,-211976.0,-27231.0,4077.415500,-27071.78635
3,2022-12-31,AMMO.VAREJO,-46288.82802,0.483290,8624.494377,0.0,380609.3204,-196664.5365,183944.7840,-263660.5632,-182494.0,-40999.0,9472.187080,-49639.75025


In [15]:
import pandas as pd

complete_df = pd.concat([df_pivot, sheets_data]).reset_index(drop=True)

complete_df.head()

,DT_FIM_EXERC,DENOM_CIA,Custo de Bens/Serviços,Depreciações e Amortizações_1,Depreciações e Amortizações_2,Despesas Gerais e Administrativas,Despesas de Vendas,Despesas/Receitas Operacionais (totais),EBIT,Outras Despesas Operacionais,Outras Receitas Operacionais,Perdas por não recuperabilidade de Ativos,Receita de Venda de Bens/Serviços,Resultado Bruto,Calculo_Margem_Bruta
0,2010-12-31,ALPARGATAS S.A.,-1217751.0,50771.0,0.0,-120391.0,-546472.0,-697180.0,324225.0,-46457.0,25362.0,0.0,2239156.0,1021405.0,NaN
1,2010-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,-339884.0,0.0,0.0,-45679.0,-96597.0,-138821.0,92820.0,0.0,3455.0,0.0,571525.0,231641.0,NaN
2,2010-12-31,GUARARAPES CONFECCOES S.A.,-1115401.0,0.0,108898.0,-363957.0,-679362.0,-1037539.0,455044.0,0.0,5780.0,0.0,2607984.0,1492583.0,NaN
3,2010-12-31,KARSTEN S.A.,-230175.0,0.0,5579.0,-26464.0,-76622.0,-94873.0,32883.0,0.0,8213.0,0.0,357931.0,127756.0,NaN
4,2010-12-31,LOJAS RENNER S.A.,-1209459.0,75737.0,0.0,-211764.0,-641702.0,-1137405.0,404474.0,-209378.0,13611.0,-88172.0,2751338.0,1541879.0,NaN


In [16]:
# 1 Calculo_depre = `Depreciações e Amortizações_2`+`Depreciações e Amortizações_1`
complete_df["Calculo_depre"] = (
    complete_df["Depreciações e Amortizações_2"] + complete_df["Depreciações e Amortizações_1"]
)

# 2 Calculo_EBITDA=`EBIT`+`Depreciações e Amortizações_1`+`Depreciações e Amortizações_2`
complete_df["Calculo_EBITDA"] = complete_df["EBIT"] + complete_df["Calculo_depre"]

# 3 Calculo_Margem_Bruta =`Resultado Bruto`/`Receita de Venda de Bens/Serviços`
complete_df["Calculo_Margem_Bruta"] = complete_df["Resultado Bruto"] / complete_df["Receita de Venda de Bens/Serviços"]

# 4 Calculo_Margem_EBITDA =`Calculo_EBITDA`/`Receita de Venda de Bens/Serviços`
complete_df["Calculo_Margem_EBITDA"] = complete_df["Calculo_EBITDA"] / complete_df["Receita de Venda de Bens/Serviços"]

# 5 Calculo_Margem_SGeA_Total =`Despesas/Receitas Operacionais (totais)`/`Receita de Venda de Bens/Serviços`
complete_df["Calculo_Margem_SGeA_Total"] = (
    complete_df["Despesas/Receitas Operacionais (totais)"] / complete_df["Receita de Venda de Bens/Serviços"]
)

# 6 Calculo_Margem_SGeA_Administrativs =`Despesas Gerais e Administrativas`/`Receita de Venda de Bens/Serviços`
complete_df["Calculo_Margem_SGeA_Administrativs"] = (
    complete_df["Despesas Gerais e Administrativas"] / complete_df["Receita de Venda de Bens/Serviços"]
)

# 7 Calculo_Margem_SGeA_Vendas=`Despesas de Vendas`/`Receita de Venda de Bens/Serviços`
complete_df["Calculo_Margem_SGeA_Vendas"] = (
    complete_df["Despesas de Vendas"] / complete_df["Receita de Venda de Bens/Serviços"]
)

# 8 Calculo_Margem_Contribuição=(`Receita de Venda de Bens/Serviços`+`Custo de Bens/Serviços`
# +`Despesas de Vendas`)/`Receita de Venda de Bens/Serviços`
complete_df["Calculo_Margem_Contribuição"] = (
    complete_df["Receita de Venda de Bens/Serviços"]
    + complete_df["Custo de Bens/Serviços"]
    + complete_df["Despesas de Vendas"]
) / complete_df["Receita de Venda de Bens/Serviços"]

# 9 Receita_para_BreakEven=`Despesas Gerais e Administrativas`/`Calculo_Margem_Contribuição`
complete_df["Receita_para_BreakEven"] = (
    complete_df["Despesas Gerais e Administrativas"] / complete_df["Calculo_Margem_Contribuição"]
)

complete_df["DT_YEAR"] = complete_df["DT_FIM_EXERC"].apply(lambda x: x[:4])

In [18]:
complete_df.shape

(125, 24)

In [23]:
complete_df["Resultado Bruto"].astype(float) / complete_df["Receita de Venda de Bens/Serviços"].astype(float)

0      0.456156
1      0.405303
2      0.572313
3      0.356929
4      0.560411
         ...   
120    0.656715
121    0.488221
122    0.514921
123    0.533911
124    0.483290
Length: 125, dtype: float64

In [24]:
complete_df

,DT_FIM_EXERC,DENOM_CIA,Custo de Bens/Serviços,Depreciações e Amortizações_1,Depreciações e Amortizações_2,Despesas Gerais e Administrativas,Despesas de Vendas,Despesas/Receitas Operacionais (totais),EBIT,Outras Despesas Operacionais,Outras Receitas Operacionais,Perdas por não recuperabilidade de Ativos,Receita de Venda de Bens/Serviços,Resultado Bruto,Calculo_Margem_Bruta,Calculo_depre,Calculo_EBITDA
0,2010-12-31,ALPARGATAS S.A.,-1217751.0,50771.0,0.0,-120391.0,-546472.0,-697180.0,324225.0,-46457.0,25362.0,0.0,2239156.0,1021405.0,NaN,50771.0,374996.0
1,2010-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,-339884.0,0.0,0.0,-45679.0,-96597.0,-138821.0,92820.0,0.0,3455.0,0.0,571525.0,231641.0,NaN,0.0,92820.0
2,2010-12-31,GUARARAPES CONFECCOES S.A.,-1115401.0,0.0,108898.0,-363957.0,-679362.0,-1037539.0,455044.0,0.0,5780.0,0.0,2607984.0,1492583.0,NaN,108898.0,563942.0
3,2010-12-31,KARSTEN S.A.,-230175.0,0.0,5579.0,-26464.0,-76622.0,-94873.0,32883.0,0.0,8213.0,0.0,357931.0,127756.0,NaN,5579.0,38462.0
4,2010-12-31,LOJAS RENNER S.A.,-1209459.0,75737.0,0.0,-211764.0,-641702.0,-1137405.0,404474.0,-209378.0,13611.0,-88172.0,2751338.0,1541879.0,NaN,75737.0,480211.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,2023-12-31,VESTE S.A. ESTILO,-380986.0,170928.0,0.0,-164032.0,-511336.0,-696357.0,32481.0,-25792.0,4803.0,0.0,1109824.0,728838.0,NaN,170928.0,203409.0
121,2019-12-31 00:00:00,AMMO.VAREJO,-164222.0922,0,4575.11993,-43578,-129405,-182044.786,15063.47676,-10053.15228,991.3662774,NaN,320885.0801,156662.9879,0.4882214774,4575.119930,15063.476764575.119930
122,2020-12-31 00:00:00,AMMO.VAREJO,-209138.3288,0,6676.01749,-23646,-188015,-226511.9087,11067.79964,-16815.83439,1964.925644,NaN,431143.2593,222004.9304,0.5149214922,6676.017490,11067.799646676.017490
123,2021-12-31 00:00:00,AMMO.VAREJO,-230830.9053,0,8516.20133,-27231,-211976,-262201.3709,20743.719,-27071.78635,4077.4155,NaN,495250.3615,264419.4562,0.5339106778,8516.201330,20743.7198516.201330


In [1]:
from peers_comparison.dfp_data_retriever import DFPDataRetriever

companies_cvm_codes = [22349, 25208, 25011, 25518, 8133, 10456, 24694, 4669, 22055, 24848, 26506, 21440, 10324, 4146]
type_docs = ["BPA", "BPP", "DFC_MD", "DFC_MI", "DMPL", "DRE", "DVA"]
first_year = 2010
last_year = 2024

retriever = DFPDataRetriever()

retriever.get_dfp_data(
    companies_cvm_codes=companies_cvm_codes, type_docs=type_docs, first_year=first_year, last_year=last_year
)
retriever.create_report()

retriever.dfp_report
# retriever.dfp_original_data

,DT_FIM_EXERC,CD_CVM,CNPJ_CIA,DENOM_CIA,CD_CONTA,DESC_CONTA,VL_CONTA
5427,2010-12-31,4669,08.402.943/0001-52,GUARARAPES CONFECCOES S.A.,6.01.01.04,Depreciações e Amortizações_2,1.088980e+08
9697,2010-12-31,21440,49.669.856/0001-43,VESTE S.A. ESTILO,6.01.01.02,Depreciações e Amortizações_1,1.317200e+07
11325,2010-12-31,10456,61.079.117/0001-05,ALPARGATAS S.A.,6.01.01.02,Depreciações e Amortizações_1,5.077100e+07
11558,2010-12-31,22055,61.189.288/0001-89,MARISA LOJAS S.A.,6.01.01.02,Depreciações e Amortizações_1,1.033550e+08
12767,2010-12-31,4146,82.640.558/0001-04,KARSTEN S.A.,6.01.01.04,Depreciações e Amortizações_2,5.579000e+06
...,...,...,...,...,...,...,...
15256,2023-12-31,8133,92.754.738/0001-62,LOJAS RENNER S.A.,3.04.02,Despesas Gerais e Administrativas,-1.726176e+09
15257,2023-12-31,8133,92.754.738/0001-62,LOJAS RENNER S.A.,3.04.03,Perdas por não recuperabilidade de Ativos,-1.343277e+09
15258,2023-12-31,8133,92.754.738/0001-62,LOJAS RENNER S.A.,3.04.04,Outras Receitas Operacionais,1.987970e+08
15259,2023-12-31,8133,92.754.738/0001-62,LOJAS RENNER S.A.,3.04.05,Outras Receitas Operacionais,-8.027150e+08


In [4]:
import pandas_gbq

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "projetoomni"

# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
table_id = "cvm_peers_reports.dfp_report"

pandas_gbq.to_gbq(retriever.dfp_report, table_id, project_id=project_id)

In [2]:
retriever.dfp_report["DT_FIM_EXERC"]

5427    2010-12-31
9697    2010-12-31
11325   2010-12-31
11558   2010-12-31
12767   2010-12-31
           ...    
15256   2023-12-31
15257   2023-12-31
15258   2023-12-31
15259   2023-12-31
15261   2023-12-31
Name: DT_FIM_EXERC, Length: 1175, dtype: datetime64[ns]

In [3]:
# !pip freeze